# Precisão (interativo)

Soluções numéricas e analíticas da equação para o decaimento radioativo de um material

$$\frac{du}{dt}=-au$$

Abaixo você pode ver como o espaçamento dos pontos no tempo afeta o resultado da solução para o caso de 

$$u(t_0=0)=1$$

No intervalo de tempo

$$t=[0,1000)$$

Abaixo você vê um aplicativo interativo comparando as soluções numéricas utilizando: método de Euler (também conhecido como Runge-Kutta de ordem 1), Runge-Kutta de ordem 2 e Runge-Kutta de ordem 4.

In [20]:
%matplotlib widget
from ipywidgets import interact
import ipywidgets as wid
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

plt.rc('font',size=12)
plt.rc('lines',markersize=5,linewidth=2)
a=0.005
dt=0.1
tt=np.arange(0,1000,dt)
lay=wid.Layout(width="repeat(auto-fill, 180px)")
laybox=wid.Layout(width="95%")
layauto=wid.Layout(width="25%")
dt=wid.FloatSlider(value=50,min=1.0,
    max=200,step=5,
    description='$\Delta t:$',
    continuous_update=False,layout=laybox)
an=wid.ToggleButton(
    value=True,
    description='Analítico',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Ativa ou desativa a solução analítica',
    icon='check',layout=layauto)
e=wid.ToggleButton(
    value=True,
    description='RK1(Euler)',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Ativa ou desativa a solução RK1(Euler)',
    icon='check',layout=layauto)
r2=wid.ToggleButton(
    value=True,
    description='RK2',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Ativa ou desativa a solução RK2',
    icon='check',layout=layauto)
r4=wid.ToggleButton(
    value=True,
    description='RK4',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Ativa ou desativa a solução RK4',
    icon='check',layout=layauto)

Hsol=wid.HBox([an,e,r2,r4],layout=lay)
Vmain=wid.VBox([dt,Hsol],layout=laybox)

def main(dt,an,e,r2,r4):
    fig, ax=plt.subplots()
    t=np.arange(0,1000,dt)
    nt=t.size
    ue=np.zeros((nt))
    ur2=np.zeros((nt))
    ur4=np.zeros((nt))
    ue[0], ur2[0], ur4[0] = (1.0,1.0,1.0)
    
    @njit
    def Fl(fv):
        return -a*fv
    
    @njit
    def euler(f,dt):
        for i in range(0,nt-1):
            f[i+1]=f[i]+Fl(f[i])*dt
        return f
    
    @njit
    def rk2(f,dt):
        for i in range(0,nt-1):
            k1=Fl(f[i])
            k2=Fl(f[i]+dt*k1)
            f[i+1]=f[i]+dt*(k1+k2)/2
        return f
    
    @njit
    def rk4(f,dt):
        for i in range(0,nt-1):
            k1=Fl(f[i])
            k2=Fl(f[i]+k1*dt/2)
            k3=Fl(f[i]+k2*dt/2)
            k4=Fl(f[i]+k3*dt)
            f[i+1]=f[i]+dt*(k1+2*k2+2*k3+k4)/6
        return f
    
    plt.clf()
    if e==True: ue=euler(ue,dt)
    if r2==True: ur2=rk2(ur2,dt)
    if r4==True: ur4=rk4(ur4,dt)
    if (e,an,r2,r4)!=(False,False,False,False):
        if an==True: fig=plt.plot(tt,np.exp(-a*tt),
                        'k-',linewidth=6,label='Analítico')
        if r4==True: fig=plt.plot(t,ur4,
                        'o--',color='skyblue',label=f'RK4 (NT={nt})')
        if r2==True: fig=plt.plot(t,ur2,
                        'o--',color='fuchsia',label=f'RK2 (NT={nt})')
        if e==True: fig=plt.plot(t,ue,
                        'o--',color='coral',label=f'Euler (NT={nt})')   
        ax.set_xlabel('Tempo')
        ax.set_ylabel('Massa do elemento')
        plt.title(f'$\Delta t=${dt:.3f}')
        leg=plt.legend()
    else:
        plt.clf()
        plt.title('Nada para plotar')
    
out=wid.interactive_output(main, {'dt':dt,'an':an,'e':e,'r2':r2,'r4':r4})
display(Vmain,out)
fig.canvas.layout.width = '100%'
fig.canvas.layout.heigth = '100%'

Output()